# Data extraction from the title page of the thesis

## getting main page

In [3]:
# pdftotext
import pdftotext

# pandas to work with csv
import pandas as pd

#os to work with file system
from os import listdir
from os.path import isfile, join

# regex
import re

In [4]:
def get_text_from_main_page_pdftotext(filename: str) -> str:
    with open(filename, 'rb') as f:
        pdf = pdftotext.PDF(f)
        main_page = pdf[0]
        
        # look at second page if there is some info 
        second_page = re.findall(r"(?i)^([\s\S]*)оглавление|содержание", pdf[1])
        if len(second_page) > 0 :
            main_page += second_page[0]
#             print(second_page)

        # look at third page if there is some info 
        else:
            third_page = re.findall(r"(?i)^([\s\S]*)оглавление|содержание", pdf[2])
            if len(third_page) > 0 :
                main_page += pdf[1]        # add second too
                main_page += third_page[0]
#                 print(main_page)
            
        return main_page

In [5]:
# # list all files
# path = "diploms"
# files = []
# for folder in listdir(path):
#     files += [join(path, folder, file) for file in listdir(join(path, folder)) if isfile(join(path, folder, file))]

# data = pd.DataFrame(columns = ["filename", "text"])

# for filename in files:
#     try:
#         text = get_text_from_main_page_pdftotext(filename)
#         data = data.append(pd.Series(
#             [filename, text], index = data.columns
#         ), ignore_index = True)
#         print(f'\x1B[32mSuccess \x1B[0m- {filename}')
#     except Exception as e:
#         print(f'\x1B[31mFailed \x1B[0m- {filename}\x1B[31m', e)

# data.to_csv("diplomas_pdftotext.csv", index = False)

the dataset is ready at `diplomas_pdftotext.csv`

## extracting

In [6]:
df = pd.read_csv("diplomas_pdftotext.csv")
df

,filename,text
0,diploms/MFTI/2017MS_Bubis.pdf,NaN
1,diploms/MFTI/2016MS_Bolotskaya.pdf,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ ОБРАЗОВАТЕЛЬНОЕ ...
2,diploms/MFTI/2017MS_Gagkaeva.pdf,NaN
3,diploms/MFTI/2016MS_Shuravin.pdf,Министерство образования и науки Российско...
4,diploms/MFTI/2016BS_Galiullin.pdf,Ìîñêîâñêèé ôèçèêî-òåõíè÷åñêèé èíñòèòóò (ãîñóäà...
...,...,...
105,diploms/Rau/Копия Дипломная работа - Мкртчян М...,ГОУ ВПО РОССИЙСКО-АРМЯНСКИЙ (СЛАВЯНСКИЙ)\n ...
106,diploms/Rau/Копия Гюласарян Андраник Эдуардови...,РОССИЙСКО - АРМЯНСКИЙ (СЛАВЯНСКИЙ)...
107,diploms/Rau/Копия Саргсян Э. Diplom.pdf,ГОУ ВПО РОССИЙСКО-АРМЯНСКИЙ\n ...
108,diploms/Rau/Копия Саакян Мигран (2).pdf,DocuSign Envelope ID: 3AACB8B0-654A-48D0-B544-...


In [7]:
from my_metadata import get_metadata
from my_metadata import rau_metadata, mipt_metadata, rudn_metadata, hse_metadata, msu_metadata, any_metadata

import traceback

In [8]:
import re

def get_metadata(title_page : str) -> tuple:
    if type(title_page) != str or len(title_page) == 0:
        return None, None, None, None, None, None, None, None
    
    year = re.findall(r'(2[0-9]{3})', title_page)
    year = year[-1] if year else None
    
#     people = re.findall(r"([A-Я][а-я]+ [A-Я][а-я]+ [A-Я][а-я]+)|([А-Я]. ?[А-Я]. ?[A-Я][а-я]+)", title_page)
#     print(people)
    
    if re.search(r"(?i)Российско ?- ?армянский", title_page):
        university = 'rau'
        faculty, department, speciality, author, supervisor, title = rau_metadata(title_page)
        
    elif re.search(r"(?i)московский физико[\s\n]?-[\s\n\t]*технический институт", title_page):
        university = 'mipt'
        faculty, department, speciality, author, supervisor, title = mipt_metadata(title_page)
        
    elif re.search(r"(?i)Высшая школа экономики", title_page):
        university = 'hse'
        faculty, department, speciality, author, supervisor, title = hse_metadata(title_page)
        
    elif re.search(r"(?i)дружбы народов", title_page):
        university = 'rudn'
        faculty, department, speciality, author, supervisor, title = rudn_metadata(title_page)
        
    elif (re.search(r"(?i)московский государственный", title_page)):
        university = 'msu'
        faculty, department, speciality, author, supervisor, title = msu_metadata(title_page)
    
    # try to extract something
    else:
        university = re.findall("(?i)$(.*)университет", title_page)
        faculty, department, speciality, author, supervisor, title = any_metadata(title_page)
        
    return year, university, faculty, department, speciality, author, supervisor, title


def splitted_metadata(title_page : str) -> tuple:
    author = None 
    supervisor = None
    title = None
    
    # split in words
    tokenized = re.split(r'\s', title_page)

    for i in range(len(tokenized)):
        if re.match(r"(?i)студент", tokenized[i]) or re.match(r"(?i)исполнитель", tokenized[i]):
            for j in range(i+1, len(tokenized)):
                if tokenized[j] and re.match(r"^[А-Я].*", tokenized[j]) != None:
                    author = tokenized[j]
#                     print("~", author)
                    if tokenized[j+1] and re.match(r"^[А-Я].*", tokenized[j+1]) != None:
                        author += ' ' + tokenized[j+1]
#                         print("~", author)
                        if tokenized[j+2] and re.match(r"^[А-Я].*", tokenized[j+2]) != None:
                            author += ' ' + tokenized[j+2]
#                             print("~", author)
                if author:
                    break
                            
#             print("author:", author)

        if re.match(r"(?i)руководитель", tokenized[i]):
            for j in range(i+1, len(tokenized)):
                if tokenized[j] and re.match(r"^[А-Я].*", tokenized[j]) != None:
                    supervisor = tokenized[j]
#                     print("~", supervisor)
                    if tokenized[j+1] and re.match(r"^[А-Я].*", tokenized[j+1]) != None:
                        supervisor += ' ' + tokenized[j+1]
#                         print("~", supervisor)
                        if tokenized[j+2] and re.match(r"^[А-Я].*", tokenized[j+2]) != None:
                            supervisor += ' ' + tokenized[j+2]
#                             print("~", supervisor)
                if supervisor:
                    break
#             print("supervisor:", supervisor)
            
            
        if re.match(r"(?i)тема", tokenized[i]):
            title = ""
            i += 1
            if(tokenized[i] and tokenized[i][0] in ('"', "«", '“', '“')):
                while(tokenized[i][-1] not in ('"', "»", '”', ".")):
                    title += ' ' + tokenized[i]
                    i += 1
                    while not tokenized[i]:
                        i += 1
#                         print('hm', tokenized[i])
                        
                title += ' ' + tokenized[i]
            else:
                while tokenized[i]:
                    title += ' ' + tokenized[i]
                    i += 1
            title = ' '.join(title.split())
            
#     print(title_page)
    return author, supervisor, title


# RAU
def rau_metadata(title_page : str) -> tuple:
    faculty = "ПМИ"   # as there are no faculty in diploma mentioned
    department = re.findall(r"(?i)Кафедра.*", title_page)
    department = department[0].lower() if department else None
    
    speciality = re.findall(r"(?i)Специальность:\s*(.*)", title_page)
    speciality = speciality[0].lower() if speciality else None
    
    author, supervisor, title = splitted_metadata(title_page)
    
    return faculty, department, speciality, author, supervisor, title


def mipt_metadata(title_page : str) -> tuple:
    faculty = re.findall(r"(?i)Факультет .*", title_page)
    faculty = faculty[0].lower() if faculty else None
        
    department = re.findall(r"(?i)Кафедра.*", title_page)
    department = department[0].lower() if department else None
    
    speciality = re.findall(r"[0-9]{2}.[0-9]{2}.[0-9]{2}", title_page)
    speciality = speciality[0] if speciality else None
    
    author = re.findall(r"C?c?тудент.*(([А-Я]. ?)?[А-Я]. ?[A-Я][а-я]+)|([A-Я][а-я]+ [A-Я][а-я]+ [A-Я][а-я]+)", title_page)
    supervisor = re.findall(r"Научный руководитель.*([А-Я]. ?[А-Я]. [A-Я][а-я]+)", title_page)
    author, supervisor, title = splitted_metadata(title_page)
#     title = re.findall(r"«(.*)»", title_page)
    
    return faculty, department, speciality, author, supervisor, title


def rudn_metadata(title_page : str) -> tuple:
    faculty = re.findall(r"(?i)учебный институт: (.*)", title_page)
    if not faculty:
        faculty = re.findall(r"(?i)\s+(.*)\sинститут", title_page)
    if faculty:
        faculty = faculty[0].lower()
    
    department = re.findall(r"(?i)Кафедра.*", title_page)
    department = department[0] if department else None
    
    speciality = re.findall(r"[0-9]{2}.[0-9]{2}.[0-9]{2}", title_page)
    speciality = speciality[0]  if speciality else None
        
#     author = None
#     supervisor = None
    author, supervisor, title = splitted_metadata(title_page)

    title = re.findall(r"(?i)ТЕМА(.?)", title_page)
    
    return faculty, department, speciality, author, supervisor, title


def hse_metadata(title_page : str) -> tuple:
    faculty = re.findall(r"(?i)Факультет.*", title_page)
    if faculty:
        faculty = faculty[0].lower()
    department = None
    
    speciality = re.findall(r"[0-9]{2}.[0-9]{2}.[0-9]{2}", title_page)
    speciality = speciality[0]  if speciality else None
    
    author, supervisor, title = splitted_metadata(title_page)
    
    return faculty, department, speciality, author, supervisor, title


def msu_metadata(title_page : str) -> tuple:
    faculty = re.findall(r"(?i)\s*(.*Факультет.*)", title_page)
    faculty = faculty[0].lower() if faculty else None
    
    department = re.findall(r"(?i)Кафедра.*", title_page)
    department = department[0].lower() if department else None
    
    speciality = None
    
    author, supervisor, title = splitted_metadata(title_page)
    title = re.findall(r"(?i)Тема:\s*(.*)", title_page)
    
    return faculty, department, speciality, author, supervisor, title


def any_metadata(title_page : str) -> tuple:
    faculty = re.findall(r"(?i)\s*(.*Факультет.*)", title_page)
    faculty = faculty[0].lower() if faculty else None
    
    department = re.findall(r"(?i)Кафедра.*", title_page)
    department = department[0].lower() if department else None
    
    speciality = re.findall(r"[0-9]{2}.[0-9]{2}.[0-9]{2}", title_page)
    speciality = speciality[0]  if speciality else None
    
    author, supervisor, title = splitted_metadata(title_page)

    return faculty, department, speciality, author, supervisor, title


In [9]:
years = []
universities = []
faculties = []
departments = []
specialities = []
authors = []
supervisors = []
titles = []
for j in range(df.shape[0]):
    try:
        t = get_metadata(df.iloc[j].text)
        year, university, faculty, department, speciality, author, supervisor, title = t
        years.append(year)
        universities.append(university)
        faculties.append(faculty)
        departments.append(department)
        specialities.append(speciality)
        authors.append(author)
        supervisors.append(supervisor)
        titles.append(title)
    except Exception as e:
        print(j, df.iloc[j].title, e)
        traceback.print_exc(e)
        continue

In [10]:
df["year"] = years
df["university"] = universities
df["faculty"] = faculties
df["department"] = departments
df["speciality"] = speciality
df["author"] = authors
df["supervisors"] = supervisors
df["title"] = titles

In [11]:
df

,filename,text,year,university,faculty,department,speciality,author,supervisors,title
0,diploms/MFTI/2017MS_Bubis.pdf,NaN,None,None,None,None,прикладная математика и информатика,None,None,None
1,diploms/MFTI/2016MS_Bolotskaya.pdf,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ ОБРАЗОВАТЕЛЬНОЕ ...,2016,mipt,факультет общей и прикладной фикики,кафедра физики и технологии наноструктур,прикладная математика и информатика,None,Юрченко Станислав Олегович,None
2,diploms/MFTI/2017MS_Gagkaeva.pdf,NaN,None,None,None,None,прикладная математика и информатика,None,None,None
3,diploms/MFTI/2016MS_Shuravin.pdf,Министерство образования и науки Российско...,2016,mipt,факультет общей и прикладной физики,кафедра физики и технологии наноструктур,прикладная математика и информатика,Шуравина Никиты Сергеевича,Долганов П.В.,None
4,diploms/MFTI/2016BS_Galiullin.pdf,Ìîñêîâñêèé ôèçèêî-òåõíè÷åñêèé èíñòèòóò (ãîñóäà...,2016,[],None,None,прикладная математика и информатика,None,None,None
...,...,...,...,...,...,...,...,...,...,...
105,diploms/Rau/Копия Дипломная работа - Мкртчян М...,ГОУ ВПО РОССИЙСКО-АРМЯНСКИЙ (СЛАВЯНСКИЙ)\n ...,2020,rau,ПМИ,кафедра математики и математической кибернетики,прикладная математика и информатика,Мкртчян Метаксия Арсеновна,Арамян Рафик Грачикович,“Вычисление страховых премий для некоторых групп”
106,diploms/Rau/Копия Гюласарян Андраник Эдуардови...,РОССИЙСКО - АРМЯНСКИЙ (СЛАВЯНСКИЙ)...,2020,rau,ПМИ,кафедра: математики и математического моделиро...,прикладная математика и информатика,ПМИ Гюласарян Андраник,Арутюнян К.В. Исполнитель:,“Факторизация некоторых треугольных матриц-фун...
107,diploms/Rau/Копия Саргсян Э. Diplom.pdf,ГОУ ВПО РОССИЙСКО-АРМЯНСКИЙ\n ...,2020,rau,ПМИ,кафедра математической кибернетики,прикладная математика и информатика,Э. А. Саргсян,П. А. Петросян,“О палитровом индексе некоторых графов”
108,diploms/Rau/Копия Саакян Мигран (2).pdf,DocuSign Envelope ID: 3AACB8B0-654A-48D0-B544-...,2020,rau,ПМИ,кафедра системного программирования,прикладная математика и информатика,Саакян Мигран Арамович,Авагумян Арсен,“АСИНХРОННЫЕ ФОРМЫ”


In [12]:
df.to_csv("diplomas_pdftotext_extracted.csv", index = False)

## Statistics

In [13]:
# all
df.describe()

,filename,text,year,university,faculty,department,speciality,author,supervisors,title
count,110,105,102,105,102,71,110,68,101,68
unique,110,104,8,6,14,19,1,65,88,40
top,diploms/hse/agisheva_da_subekty-sovremennogo-m...,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ АВТОНОМНОЕ ОБРАЗО...,2020,rau,ПМИ,кафедра математической кибернетики,прикладная математика и информатика,Студент,П. А. Петросян,[]
freq,1,2,39,32,32,13,110,2,4,28


In [14]:
df.isnull().sum(axis = 0)

filename        0
text            5
year            8
university      5
faculty         8
department     39
speciality      0
author         42
supervisors     9
title          42
dtype: int64

In [15]:
# msu
df[df["filename"].apply(lambda x: "msu" in x)].describe()

,filename,text,year,university,faculty,department,speciality,author,supervisors,title
count,30,30,30,30,28,24,30,18,28,30
unique,30,30,7,2,4,9,1,17,26,7
top,diploms/msu/badretdinova_l_r.pdf,Московский государственный университет имени...,2019,msu,юридический факультет,кафедра системного программирования,прикладная математика и информатика,Барановой Полины Олеговны,Игнатьева И.А.,[]
freq,1,1,9,29,16,8,30,2,2,24


In [16]:
df[df["filename"].apply(lambda x: "hse" in x)].describe()

,filename,text,year,university,faculty,department,speciality,author,supervisors,title
count,26,26,26,26,26,0,26,9,26,0
unique,26,25,5,1,6,0,1,8,24,0
top,diploms/hse/2.pdf,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ АВТОНОМНОЕ ОБРАЗО...,2021,hse,факультет социальных наук,NaN,прикладная математика и информатика,МУВ,Н. В. Большаков,NaN
freq,1,2,9,26,11,NaN,26,2,2,NaN


In [17]:
df[df["filename"].apply(lambda x: "ruden" in x)].describe()

,filename,text,year,university,faculty,department,speciality,author,supervisors,title
count,6,6,6,6,6,6,6,3,6,6
unique,6,6,3,1,1,3,1,3,6,2
top,diploms/ruden/dordzhi-goryaeva_a__c.pdf,МИНИСТЕРСТВО ОБРАЗОВАНИЯ И НАУКИ РОССИЙСКОЙ...,2016,rudn,юридический,Кафедра гражданского и трудового права,прикладная математика и информатика,Горбачева Анна Игоревна,А.Н. Кирсанов,[]
freq,1,1,3,6,6,4,6,1,1,4


In [18]:
df[df["filename"].apply(lambda x: "MFTI" in x)].describe()

,filename,text,year,university,faculty,department,speciality,author,supervisors,title
count,16,11,11,11,10,9,16,7,9,1
unique,16,11,2,2,2,1,1,7,9,1
top,diploms/MFTI/2017BS_Grebenchuk.pdf,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ ОБРАЗОВАТЕЛЬНОЕ ...,2016,mipt,факультет общей и прикладной физики,кафедра физики и технологии наноструктур,прикладная математика и информатика,А.А. Худякова,Доктор,Разработка однофотонного детектора в
freq,1,1,10,10,9,9,16,1,1,1


In [19]:
df[df["filename"].apply(lambda x: "Rau" in x)].describe()

,filename,text,year,university,faculty,department,speciality,author,supervisors,title
count,32,32,29,32,32,32,32,31,32,31
unique,32,32,1,1,1,7,1,31,26,31
top,diploms/Rau/Копия Асатрян Нарек Арамович.pdf,ГОУ ВПО РОССИЙСКО-АРМЯНСКИЙ (СЛАВЯНС...,2020,rau,ПМИ,кафедра математической кибернетики,прикладная математика и информатика,ПМИ Гюласарян Андраник,П. А. Петросян,“АСИНХРОННЫЕ ФОРМЫ”
freq,1,1,29,32,32,13,32,1,4,1
